In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("공공/훈련데이터셋.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 181408 entries, 0 to 181407
Data columns (total 9 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   지역코드           181408 non-null  int64  
 1   최저기온(°C)       181408 non-null  float64
 2   3.0m 지중온도(°C)  181408 non-null  float64
 3   평균 현지기압(hPa)   181408 non-null  float64
 4   가조시간(hr)       181408 non-null  float64
 5   평균 상대습도(%)     181408 non-null  float64
 6   풍정합(100m)      181408 non-null  float64
 7   합계 소형증발량(mm)   181408 non-null  float64
 8   파워             181408 non-null  float64
dtypes: float64(8), int64(1)
memory usage: 12.5 MB


In [9]:
from sklearn.model_selection import train_test_split

# feature / target
y = df["파워"]
X = df.drop(columns=["파워"])

# train / validation split
X_train, X_val, y_train, y_val = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42
)

In [10]:
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

# target
y = df["파워"]

# feature
X = df.drop(columns=["파워"])

# CatBoost가 자동으로 처리할 수 있게 int/object 유지
cat_features = ["지역코드"]  # 컬럼명 정확히 맞춰줘

model = CatBoostRegressor(
    iterations=500,
    depth=6,
    learning_rate=0.05,
    loss_function="RMSE",
    random_seed=42,
    verbose=100
)

model.fit(
    X_train, y_train,
    cat_features=cat_features,
    eval_set=(X_val, y_val),
    use_best_model=True
)

# 예측
y_pred = model.predict(X_val)

rmse = np.sqrt(mean_squared_error(y_val, y_pred))
r2 = r2_score(y_val, y_pred)

print(f"RMSE: {rmse}")
print(f"R2: {r2}")

0:	learn: 337711.2955175	test: 346412.3753293	best: 346412.3753293 (0)	total: 194ms	remaining: 1m 36s
100:	learn: 304160.8675200	test: 312350.8082007	best: 312350.8082007 (100)	total: 3.48s	remaining: 13.8s
200:	learn: 297012.0346750	test: 305558.7191975	best: 305558.7191975 (200)	total: 6.83s	remaining: 10.2s
300:	learn: 291497.6546909	test: 300693.2841080	best: 300693.2841080 (300)	total: 10.1s	remaining: 6.69s
400:	learn: 284486.8434327	test: 293547.7398220	best: 293547.7398220 (400)	total: 13.1s	remaining: 3.24s
499:	learn: 276843.1284409	test: 286564.4897492	best: 286564.4897492 (499)	total: 16.1s	remaining: 0us

bestTest = 286564.4897
bestIteration = 499

RMSE: 286564.48990458355
R2: 0.3266292502345015
